In [1]:
from argparse import ArgumentParser
import os
import sys
import time

import dask.dataframe as dd
import dask_geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import shapely
import xarray as xr

# from matplotlib import colors
from pandarallel import pandarallel
from scipy import stats

sys.path.insert(1, '../scripts/')
from reaches import *
from utils import *

### Parse arguments

In [2]:
# FOR NOW, SET
width_set = 'mean'

# Control flow
if width_set == 'mean':
    width = 'WidthM'
    binn = 'Bin'
elif width_set == 'min':
    width = 'WidthM_Min'
    binn = 'Bin_Min'
elif width_set == 'max':
    width = 'WidthM_Max'
    binn = 'Bin_Max'
else:
    print('Invalid width option specified, exiting.')
    # sys.exit()

In [3]:
huc2 = '15' # '01' ### SET THIS
data_path = '/nas/cee-water/cjgleason/data/SWOT/PIXC_v2_0_HUC2_' + huc2
# save_dir =

### Pixel Cloud

In [4]:
# Get job index
# slurm = int(os.environ['SLURM_ARRAY_TASK_ID'])
index = 2517 # 106

In [5]:
mdata_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/' ## HERE
file_path = os.path.join(mdata_path, 'PIXC_v2_0_HUC2_' + huc2 + '_filtered.json') ## HERE
data = open_json(file_path)

In [6]:
file_name = data[index]

In [7]:
# Get data for this tile
granule_name = file_name[:-3]
tile_name = file_name[20:28]
pass_num = int(file_name[20:23])

print(granule_name)

SWOT_L2_HR_PIXC_009_511_219L_20240122T092851_20240122T092902_PGC0_01


#### Read in PIXC

In [8]:
# Set PIXC filepath
pixc_path = os.path.join(data_path, file_name)

In [9]:
# Read in pixel group
ds_PIXC = xr.open_dataset(filename_or_obj=pixc_path,
                          group='pixel_cloud', engine='h5netcdf')

In [10]:
# Set desired data vars

## HERE CLEAN UP

variables = ['cross_track', 'water_frac',
             'pixel_area', 'height', 'geoid', 'solid_earth_tide', ## HERE
             'load_tide_fes', 'pole_tide', 'prior_water_prob', ## HERE
             # 'phase_noise_std', 'dheight_dphase', ## HERE
             'classification_qual', 'geolocation_qual', ## HERE
             'classification']

In [11]:
def bitwiseMask(ds):
    '''
    This function masks a PIXC granules: for now, it ony remove pixels
    with land classification and those with bad geolocation_qual.
    # See page 65 of PIXC PDD: https://podaac.jpl.nasa.gov/SWOT?tab=datasets-information&sections=about%2Bdata
    '''
    # Fow now, eliminate the bad stuff
    mask = np.where((ds.classification > 1) & 
                    (ds.interferogram_qual < 2**16) & (ds.classification_qual < 2**16) & ## HERE
                    (ds.geolocation_qual < 2**16) & (ds.sig0_qual < 2**16) &  ## HERE
                    (np.abs(ds.cross_track) > 10000) & (np.abs(ds.cross_track) < 60000))[0]
    
    print(mask.shape)
    return mask

In [12]:
# Make mask
mask = bitwiseMask(ds_PIXC)

if mask.shape[0] == 0:
    print('This granule has no pixels after masking, exiting.')
    # sys.exit(1)    

(597484,)


In [13]:
# Convert PIXC to GeoDataFrame
gdf_PIXC = makeGDF(ds=ds_PIXC, mask=mask, data_vars=variables)

In [14]:
del ds_PIXC

In [15]:
gdf_PIXC = gdf_PIXC.drop(columns=['latitude', 'longitude'])

In [16]:
## Flag as-in RiverSP ## HERE
gdf_PIXC['geo_qual_wse_suspect'] = 0
gdf_PIXC['geo_qual_wse_suspect'] = np.where((gdf_PIXC['geolocation_qual'] >=2**7),
                                            1, gdf_PIXC['geo_qual_wse_suspect'])

gdf_PIXC['class_qual_area_suspect'] = 0
gdf_PIXC['class_qual_area_suspect'] = np.where((gdf_PIXC['classification_qual'] >=2**7),
                                            1, gdf_PIXC['geo_qual_wse_suspect'])

gdf_PIXC = gdf_PIXC.drop(columns=['geolocation_qual', 'classification_qual'])

In [17]:
## Make indicator for wse aggregation
# (don't use lane_near_water or dark_water)
wse_klass = [3.0, 4.0, 6.0, 7.0]

gdf_PIXC['agg_wse'] = 0
gdf_PIXC.loc[gdf_PIXC['klass'].isin(wse_klass), 'agg_wse'] = 1

In [18]:
# Calculate wse
gdf_PIXC['wse'] = gdf_PIXC.height - gdf_PIXC.geoid - gdf_PIXC.solid_earth_tide - gdf_PIXC.load_tide_fes - gdf_PIXC.pole_tide
gdf_PIXC = gdf_PIXC.drop(columns=['height', 'geoid', 'solid_earth_tide', 'load_tide_fes', 'pole_tide'])

### Find correct HUC4s

In [19]:
### NHDPlus HR
## Find correct HUC4s
# Read in tile and HUC4 intersection data
dtype_dic= {'tile': str, 'huc4': str, 'coverage': float}
tile_huc4 = pd.read_csv(os.path.join(mdata_path,
                                    'huc4_swot_science_tiles.csv'),
                        dtype=dtype_dic)

In [20]:
# Make list of HUC4s that intersect the tile
huc4s = list(tile_huc4[(tile_huc4['tile'] == tile_name)]['huc4'])
# Limit to the current HUC2
huc4s = [x for x in huc4s if x.startswith(huc2)]

In [21]:
huc4s

['1501']

### Read in buffered flowlines (with extra 32 m on each side to capture full pseudo-pixels)

In [22]:
data_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_buffered/HUC2_' + huc2 + '/'
# file_paths = getFilepaths(data_path, huc2, huc4s, width_set)

In [23]:
file_paths = []

for huc in huc4s:
    file_path = data_path + 'NHDPLUS_H_' + huc + '_HU4_GDB_prepped_buffered_' + width_set + '.parquet'
    file_paths.append(file_path)

In [24]:
reach_mask = dask_geopandas.read_parquet(path=file_paths, columns=['NHDPlusID',
                                                                   width, binn,
                                                                   'Slope', 'LengthKM',
                                                                   'buffers'])

In [25]:
reach_mask = reach_mask.compute()

In [26]:
# Clip masked pixels to buffered reaches with extra width
gdf_PIXC = gpd.sjoin(gdf_PIXC, reach_mask, how='inner', predicate='within').reset_index().drop(columns=['index', 'index_right'])

In [27]:
if gdf_PIXC.shape[0] == 0:
    print('This granule has no pixels that intersect reaches, exiting.')
    # sys.exit() 

In [28]:
del reach_mask

In [29]:
# # Get NIDs of reaches with pixels in them
# ids = gdf_PIXC['NHDPlusID'].unique()

### Nadir track (a quick detour so we can delete the flowlines ASAP)

In [30]:
# Get single pixel for selecting correct nadir segment
pixel_pt = gdf_PIXC.iloc[0].geometry

In [31]:
# Find correct nadir segment and return its geometry
nadir_segment_ln = findNadir(pass_num=pass_num, pixel_pt=pixel_pt)

/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [32]:
del pixel_pt

### Find alignment

In [33]:
az_nadir = calcAzimuth(line=nadir_segment_ln)

### Find coverage

#### Make pseudo pixels

In [34]:
# Set along-track pixel resolution
azimuth_res = 22 # meters

In [35]:
pandarallel.initialize(nb_workers=int(os.environ.get('SLURM_CPUS_PER_TASK')))

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [36]:
gdf_PIXC_cov = gdf_PIXC[['points', 'pixel_area', 'geometry']].copy()

In [37]:
# Make pseudo pixels
start = time.time()
gdf_PIXC_cov['pseudo_pixel'] = gdf_PIXC_cov.parallel_apply(user_defined_function=makePseudoPixels,
                                                   args=(nadir_segment_ln,
                                                         azimuth_res),
                                                         axis=1)
end = time.time()
print(end - start)

2.830976724624634


In [38]:
del nadir_segment_ln

In [39]:
gdf_PIXC_cov = gdf_PIXC_cov.rename(columns={'geometry': 'pixel_centroid'}).set_geometry('pseudo_pixel').set_crs(epsg=3857)

In [40]:
# gdf_PIXC_cov = gdf_PIXC_cov.drop(columns='pixel_centroid')

In [41]:
# Get bounds of PIXC tile
pseudo_bounds = gdf_PIXC_cov.total_bounds

In [42]:
# Copy geometry column as sjoin will discard it
gdf_PIXC_cov['pseudo_geom'] = gdf_PIXC_cov.geometry # DO I NEED THIS

#### Read in buffered segments

In [43]:
data_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_segmented_buffered/HUC2_' + huc2 + '/'

In [44]:
file_paths = []

for huc in huc4s:
    file_path = data_path + 'NHDPLUS_H_' + huc + '_HU4_GDB_prepped_segmented_buffered_' + width_set + '.parquet'
    file_paths.append(file_path)

In [45]:
fields = ['NHDPlusID', 'counter', 'buffers']

In [46]:
segments = dask_geopandas.read_parquet(path=file_paths, columns=fields)

In [47]:
segments = segments.compute()

In [48]:
# segments = segments[segments['NHDPlusID'].isin(ids)]

In [49]:
segments = segments.clip(pseudo_bounds)

In [50]:
# Keep only reaches that are fully contained in PIXC granule
segments = segments.groupby('NHDPlusID').filter(lambda x: len(x) == 10)

In [51]:
segments = segments.sort_values(by=['NHDPlusID', 'counter']).reset_index()

In [52]:
segments = segments.drop(columns='index')

In [53]:
# # Get number of reaches per bin
# counts = pd.DataFrame(segments.sort_values(by=['NHDPlusID', 'counter'])[::10].Bin_Min.value_counts()).reset_index()

In [54]:
# counts

In [55]:
# Calculate segment area
segments['segment_area'] = segments.geometry.area

#### Join and analyze coverage

In [56]:
# Merge the segments and pseudo-puxels by intersection
sj = gpd.sjoin(segments, gdf_PIXC_cov, how='left', predicate='intersects').reset_index()

In [57]:
del segments, gdf_PIXC_cov

In [58]:
sj = sj.drop(columns=['index', 'points', 'pixel_centroid',
                      'pixel_area',
                      # 'index_old',
                      'index_right'])

In [59]:
# # NHDPlusID_left reflects the segment ids, keeping that one
# sj = sj.drop(columns=['NHDPlusID_right', 'points', 'azimuth_index',
#                       'range_index',
#                       # 'height', 'geoid',
#                       # 'klass',
#                       'latitude', 'longitude'])

In [60]:
sj = sj.set_geometry('pseudo_geom')

In [61]:
sj = sj.groupby('NHDPlusID', as_index=False).parallel_apply(user_defined_function=specialDissolve)

In [62]:
sj = sj.reset_index().drop(columns=['level_0', 'level_1'])

In [63]:
sj['pseudo_geom_clip'] = sj.parallel_apply(user_defined_function=specialClip,
                                           axis=1)

In [64]:
# Calculate the pseudo-pixel area within each node
sj['pseudo_area'] = sj.pseudo_geom_clip.area

In [65]:
sj['coverage'] = sj.pseudo_area/sj.segment_area

In [66]:
sj['coverage'] = sj['coverage'].fillna(0)

In [67]:
sj.columns

Index(['counter', 'pseudo_geom', 'NHDPlusID', 'buffers', 'segment_area',
       'pseudo_geom_clip', 'pseudo_area', 'coverage'],
      dtype='object')

In [68]:
# Drop geometry columns
sj = sj.drop(columns=['pseudo_geom', 'buffers', 'pseudo_geom_clip', 'pseudo_area'
                      , 'segment_area'])

In [69]:
sj['nodes'] = 'node_' + sj['counter'].astype(str)

In [70]:
sj = sj.drop(columns='counter')

In [71]:
sj = sj.pivot(index='NHDPlusID', columns='nodes', values='coverage').reset_index()
sj.index.name = 'index'

In [72]:
len(sj['NHDPlusID'].unique())

11968

#### Read in flowlines

In [73]:
data_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped/HUC2_' + huc2 + '/'

In [74]:
file_paths = []

for huc in huc4s:
    file_path = data_path + 'NHDPLUS_H_' + huc + '_HU4_GDB_prepped.parquet'
    file_paths.append(file_path)

In [75]:
fields = ['NHDPlusID', 'GNIS_Name', 'LengthKM',
          # 'WidthM', 'WidthM_Min',
          # 'WidthM_Max', 'Bin', 'Bin_Min', 'Bin_Max', 'StreamOrde',
          # 'Slope', 
          'geometry']

In [76]:
flowlines = dask_geopandas.read_parquet(path=file_paths, columns=fields)

In [77]:
flowlines = flowlines.compute()

In [78]:
flowlines.loc[:,'geometry'] = flowlines.geometry.explode().force_2d()

### Calculate azimuth and sinuosity (should move sinuosity to static)

In [79]:
flowlines['az_nadir'] = az_nadir

In [80]:
flowlines['temp'] = flowlines.parallel_apply(user_defined_function=calcAzSin, axis=1)

In [81]:
flowlines[['alignment', 'sinuosity']] = pd.DataFrame(flowlines['temp'].tolist(), index=flowlines.index)
flowlines = flowlines.drop(columns='temp')

In [82]:
# Rename geometry to avoid conflicts with heights
flowlines = flowlines.rename_geometry('flowlines')

In [83]:
gdf_PIXC = pd.merge(left=gdf_PIXC,
                    right=flowlines[['NHDPlusID', 'alignment',
                                     'sinuosity', 'flowlines']],
                    on='NHDPlusID')

In [84]:
del flowlines

### Calculate distance along reaches

In [85]:
def project_point(line, point):
    # Project point onto line
    return line.project(point)

In [86]:
gdf_PIXC['distance'] = gdf_PIXC.apply(lambda x: project_point(x['flowlines'], x['geometry']), axis=1)

In [87]:
# Drop geometry columns
gdf_PIXC = gdf_PIXC.drop(columns=['geometry', 'flowlines'])

### Calculate weights (skip for now)

In [88]:
# # Multiply the correct columns for the weight denominator
# gdf_PIXC['pre_denom'] = gdf_PIXC['phase_noise_std']**-2 * gdf_PIXC['dheight_dphase']**-2 * gdf_PIXC['agg_wse']

In [89]:
# len(np.unique(gdf_PIXC['NHDPlusID']))

In [90]:
# # Sum the weight denominators by reach
# denoms = gdf_PIXC.groupby(by='NHDPlusID')['pre_denom'].sum()
# denoms = denoms.rename('denom')

In [91]:
# gdf_PIXC = pd.merge(left=gdf_PIXC, right=denoms, how='left', on='NHDPlusID')
# del denoms

In [92]:
# gdf_PIXC[gdf_PIXC['NHDPlusID'] == gdf_PIXC.iloc[[0]]['NHDPlusID'][0]]

In [93]:
# gdf_PIXC['weight'] = (gdf_PIXC['phase_noise_std'] * gdf_PIXC['dheight_dphase'])**-2 * gdf_PIXC['agg_wse'] / gdf_PIXC['denom']

In [94]:
# gdf_PIXC = gdf_PIXC.drop(columns=['pre_denom', 'denom', 'phase_noise_std',
#                                   'dheight_dphase'])

In [95]:
# gdf_PIXC['wse_weighted'] = gdf_PIXC['wse'] * gdf_PIXC['weight']

In [96]:
# gdf_PIXC[['wse', 'weight', 'wse_weighted']]

In [97]:
# gdf_PIXC.groupby(by='NHDPlusID')['wse_weighted'].sum()

### Calculate slopes

In [98]:
# ?stats.linregress

In [99]:
def group_regress(group):
    
    # group = group.filter(lambda x: x['agg_wse'] == 1)
    
    if len(group) > 2:
        slope, _, r, _, se = (stats.linregress(x=group['distance'], y=group['wse']))
        return pd.Series({'swot_slope': slope, 'swot_r': r, 'swot_se': se})
    else:
        return pd.Series({'swot_slope': np.nan, 'swot_r': np.nan, 'swot_se': np.nan})

In [100]:
gdf_PIXC_for_agg = gdf_PIXC[gdf_PIXC['agg_wse'] == 1]

In [101]:
slopes = gdf_PIXC_for_agg.groupby('NHDPlusID').apply(func=group_regress, include_groups=False)

In [102]:
del gdf_PIXC_for_agg

In [103]:
gdf_PIXC = pd.merge(left=gdf_PIXC, right=slopes, how='left', on='NHDPlusID')

In [104]:
del slopes

In [105]:
gdf_PIXC = gdf_PIXC.drop(columns=['pixel_area', 'klass',
                                  # 'agg_wse',
                                  'distance'])

In [106]:
# gdf_PIXC.drop(columns=['slope', 'r', 'se'], inplace=True)
# gdf_PIXC.drop(columns = ['swot_slope_x', 'swot_r_x', 'swot_se_x',
#                          'swot_slope_y', 'swot_r_y', 'swot_se_y'],
#               inplace=True)

In [107]:
gdf_PIXC['DropM'] = gdf_PIXC['Slope'] * gdf_PIXC['LengthKM'] * 1000

In [112]:
len(gdf_PIXC.NHDPlusID.unique())

1673

In [111]:
num_pixels = gdf_PIXC.groupby('NHDPlusID').count()['points'].reset_index()
# gdf_PIXC.groupby('NHDPlusID')['points'].count().reset_index()['points']

In [ ]:
# num_wse = gdf_PIXC.groupby('NHDPlusID').count()['agg_wse'].reset_index()
temp = gdf_PIXC.groupby('NHDPlusID')['agg_wse'].value_counts().reset_index()
temp = temp[temp['agg_wse'] == 1]
temp = temp.reset_index().drop(columns=['agg_wse', 'index'])

In [123]:
gdf_PIXC = gdf_PIXC.drop(columns=['points', 'agg_wse'])

In [124]:
gdf_PIXC = pd.merge(gdf_PIXC, num_pixels, how='left', on='NHDPlusID')
gdf_PIXC = pd.merge(gdf_PIXC, temp, how='left', on='NHDPlusID')

In [127]:
gdf_PIXC = gdf_PIXC.rename(columns={'points': 'num_pixels', 'count': 'num_wse'})

In [130]:
gdf_PIXC[gdf_PIXC['prior_water_prob'] != 0]

,cross_track,water_frac,prior_water_prob,geo_qual_wse_suspect,class_qual_area_suspect,wse,NHDPlusID,WidthM,Bin,Slope,LengthKM,alignment,sinuosity,swot_slope,swot_r,swot_se,DropM,num_pixels,num_wse
14491,-22252.859375,0.283885,0.06,0,1,1114.911499,4.000050e+13,0.522455,"(0, 10]",0.047466,0.763,122.157577,1.242017,NaN,NaN,NaN,36.216604,7,2.0


In [ ]:
# ids = gdf_PIXC['NHDPlusID'].unique()

In [ ]:
# del ids

In [ ]:
test = gdf_PIXC.sort_values(by=['NHDPlusID', 'geo_qual_wse_suspect',
                                'class_qual_area_suspect', 'swot_slope'],
                            ascending=[True, False, False, False])

In [ ]:
gdf_PIXC = gdf_PIXC.drop_duplicates(subset='NHDPlusID').reset_index()

In [ ]:
gdf_PIXC[gdf_PIXC['points_y'] != gdf_PIXC['agg_wse_y']]

In [ ]:
assert gdf_PIXC.iloc[0]['points_y'] == gdf_PIXC.iloc[0]['agg_wse_y']

In [ ]:
gdf_PIXC.iloc[0]['agg_wse_y']

In [ ]:
test[test['NHDPlusID'] == 40000500183604]

In [ ]:
# gdf_PIXC[~ gdf_PIXC['swot_se'].isna()]

In [ ]:
# def checkMag(df):
#     # if df['slope_swot'] > df['Slope']:
#     #     ratio = df['slope_swot'] / df['Slope']
#     # else:
#     #     ratio = df['Slope'] / df['slope_swot']
#     if np.isnan(df['slope_swot']):
#         return False
    
#     else:
#         order1 = math.floor(math.log10(df['slope_swot']))
#         order2 = math.floor(math.log10(df['Slope']))
        
#     # if (ratio > 0.1) and (ratio < 10):
#     #     return True
#     # else:
#     #     return False

#         if order1 == order2:
#             return True
#         else:
#             return False

In [ ]:
# gdf_wse['slope_match'] = gdf_wse.apply(func = checkMag, axis=1)

In [ ]:
# slope_match = gdf_wse[gdf_wse['slope_match'] == True]['NHDPlusID'].tolist()

In [ ]:
# sj['good_wse'] = np.where(sj['NHDPlusID'].isin(slope_match), True, False)

In [ ]:
# pd.DataFrame(sj.sort_values(by=['NHDPlusID', 'counter'])[::10].Bin.value_counts()).reset_index()

In [ ]:
# reaches_min = pd.DataFrame(sj.groupby('NHDPlusID')['coverage'].min()).reset_index()

In [ ]:
# # Merge on bins
# reaches_min = pd.merge(left=reaches_min, right=sj[['NHDPlusID', 'Bin']], how='left', on='NHDPlusID')
# # Take every tenth row to get reach-level results
# reaches_min = reaches_min.sort_values(by=['NHDPlusID'])[::10].reset_index()

In [ ]:
# pd.DataFrame(reaches_min[reaches_min['coverage'] > 0.1].Bin.value_counts()).reset_index()

In [ ]:
# pd.DataFrame(sj[sj['good_wse'] == True].sort_values(by=['NHDPlusID', 'counter'])[::10].Bin.value_counts()).reset_index()

In [ ]:
# sj[sj['good_wse'] == True].iloc[50:60]

### Do stats

In [ ]:
bins = sj.Bin.unique()

#### Reaches

In [ ]:
reaches_cent, reaches_thresh, reaches_min = summarizeCoverage(df=sj, binn=binn,
                                            bins=bins, counts=counts)

In [ ]:
reaches_min

In [ ]:
reaches_min.sort_values(by=['NHDPlusID'])[::10]

In [ ]:
# d = {}
# # d_q = {}
# for i in range(1, 10):
#     threshold = i/10
#     # print(threshold)
    
#     detected = sj.groupby([binn, 'NHDPlusID'])['coverage'].apply(lambda x: (x > threshold).sum()) / 10
#     reach = detected.reset_index()
    
#     # reach = detected.groupby(binn).quantile(q=[x / 100.0 for x in range(0,100,1)]).reset_index()
        
#     d[threshold] = reach

In [ ]:
# Add a column for each DataFrame indicating the key
# for threshold, data in d.items():
#     data['threshold'] = threshold
    
for threshold, data in d.items():
    data['threshold'] = threshold

In [ ]:
# Concatenate all DataFrames into one
# reaches_desc = pd.concat(d.values())

reaches_cent = pd.concat(d.values()).rename(columns={'level_1': 'centile'})

In [ ]:
reaches_cent

In [ ]:
# reaches_cent = pd.merge(left=reaches_cent, right=counts, how='left', on=binn)

In [ ]:
reaches_cent

In [ ]:
reaches_min = pd.DataFrame(sj.groupby('NHDPlusID')['coverage'].min()).reset_index()

In [ ]:
reaches_min = pd.merge(left=reaches_min, right=sj[['NHDPlusID', binn]], how='left', on='NHDPlusID')

In [ ]:
min_cov

In [ ]:
# reaches = pd.DataFrame(data=d).T

In [ ]:
# reaches.columns = bins

### Write out

In [ ]:
save_path = os.path.join('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/', 'PIXC_v2_0_HUC2_01')

In [ ]:
# Combine node_desc
node_desc_both = pd.concat([node_desc, node_desc_w_zero], ignore_index=True)
node_desc_both

In [ ]:
# Combine node_quant
node_quant_both = pd.concat([node_quant, node_quant_w_zero], ignore_index=True)
node_quant_both

In [ ]:
# nodes_desc_both.to_csv(os.path.join(save_path, granule_name + '_nodes_describe.csv'))
# nodes_quant_both.to_csv(os.path.join(save_path, granule_name + '_nodes_quantile.csv'))

In [ ]:
# reaches_desc.to_csv(os.path.join(save_path, granule_name + '_reaches_describe.csv'))
# reaches_quant.to_csv(os.path.join(save_path, granule_name + '_reaches_quantile.csv'))

In [ ]:
test = pd.read_parquet('/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_output/PIXC_v2_0_HUC2_01_2025_03_02_min/SWOT_L2_HR_PIXC_004_242_074L_20230930T103957_20230930T104008_PGC0_01_reaches_thresh.parquet')

In [ ]:
test

### Look at heights

#### Read in buffered flowlines

In [ ]:
# data_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_buffered/HUC2_' + huc2 + '/'

In [ ]:
file_paths = []

for huc in huc4s:
    file_path = data_path + 'NHDPLUS_H_' + huc + '_HU4_GDB_prepped_buffered_' + width_set + '.parquet'
    file_paths.append(file_path)

In [ ]:
reach_extent = dask_geopandas.read_parquet(path=file_paths, columns=['NHDPlusID', 'Bin', 'Slope', 'buffers'])

In [ ]:
reach_extent = reach_extent.compute()

In [ ]:
reach_extent['area'] = reach_extent['buffers'].area